In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("parsed_logs.csv")

In [3]:
df["config"].unique()

array(['c100_res44_test_02_bn-relu6_base', 'c100_res44_test_02_bn-relu6',
       'c10_res44_test_02_relu6-bn_nanfilter',
       'c100_res44_test_02_relu6-bn_nanfilter',
       'c100_res44_test_02_relu6-bn', 'c100_res44_test_01_bn-relu_base',
       'cifar100_mobilenetv2_x1_4', 'c100_res44_test_02_gelu6_nans',
       'c10_res44_test_01_bn-relu_base', 'c10_res44_test_02_gelu6_nans',
       'c10_res44_test_02_bn-relu6_base', 'c10_res44_test_02_relu6-bn',
       'c10_res44_test_02_bn-relu6'], dtype=object)

In [4]:
configs = dict(
    # Baseline C100
    BaseC100="c100_res44_test_01_bn-relu_base.yaml",
    # "c100_res44_test_02_bn-relu6.yaml",
    # Order inversion with relu6 C100
    OrderInvC100="c100_res44_test_02_relu6-bn.yaml",
    # "c100_res44_test_02_bn-relu6_base.yaml",
    BaseBnRelu6C100="c100_res44_test_02_bn-relu6_base.yaml",
    BnRelu6C100="c100_res44_test_02_bn-relu6.yaml",
    # Order inversion + nan filter + Relu6
    OINanRelu6C100="c100_res44_test_02_relu6-bn_nanfilter.yaml",
    # Gelu and nan C100
    GeluNanC100="c100_res44_test_02_gelu6_nans.yaml",

    # Baseline C10
    BaseC10="c10_res44_test_01_bn-relu_base.yaml",
    # "c10_res44_test_02_bn-relu6.yaml",
    BnRelu6C10="c10_res44_test_02_bn-relu6.yaml",
    # Order inversion with relu6 C10
    OrderInvC10="c10_res44_test_02_relu6-bn.yaml",
    # "c10_res44_test_02_bn-relu6_base.yaml",
    BaseBnRelu6C10="c10_res44_test_02_bn-relu6_base.yaml",
    OINanRelu6C10="c10_res44_test_02_relu6-bn_nanfilter.yaml",
    # Gelu and nan C10
    GeluNanC10="c10_res44_test_02_gelu6_nans.yaml",

    MobileNetC100="cifar100_mobilenetv2_x1_4.yaml",
    MobileNetC10="cifar10_mobilenetv2_x1_4.yaml"
)
configs_inv = {v.replace(".yaml", ""): k for k, v in configs.items()}

In [5]:
configs_df = pd.DataFrame([configs]).T.reset_index().rename(columns={0:"YAML", "index": "config"})

In [6]:
configs_df

,config,YAML
0,BaseC100,c100_res44_test_01_bn-relu_base.yaml
1,OrderInvC100,c100_res44_test_02_relu6-bn.yaml
2,BaseBnRelu6C100,c100_res44_test_02_bn-relu6_base.yaml
3,BnRelu6C100,c100_res44_test_02_bn-relu6.yaml
4,OINanRelu6C100,c100_res44_test_02_relu6-bn_nanfilter.yaml
5,GeluNanC100,c100_res44_test_02_gelu6_nans.yaml
6,BaseC10,c10_res44_test_01_bn-relu_base.yaml
7,BnRelu6C10,c10_res44_test_02_bn-relu6.yaml
8,OrderInvC10,c10_res44_test_02_relu6-bn.yaml
9,BaseBnRelu6C10,c10_res44_test_02_bn-relu6_base.yaml


In [7]:
df["config"] = df["config"].apply(lambda x: configs_inv[x])

In [8]:
df["count"] = 1

In [9]:
grouped_df = df.groupby(["hostname", "config"]).sum()

In [10]:
grouped_df

it   ker_time      acc_time  ker_err  \
hostname    config                                                       
carolm20001 BaseBnRelu6C100    44976  20.427763   2929.383724   482897   
            BaseC10            52690   6.720437   2271.300349    39879   
            BaseC100           54502  13.383705   2379.867973   420120   
            BnRelu6C100        80810  26.794611   5291.821821   599096   
            GeluNanC100        30643  12.635346   2004.163502   318669   
            MobileNetC100      15205  51.839837   3043.894867    35592   
            OINanRelu6C10      19859   7.945360   1430.645173    23266   
            OINanRelu6C100     63855  17.464981   4160.807624   407952   
            OrderInvC100       44598  12.336246   2905.855822   278677   
carolp20001 BaseBnRelu6C100   369035   5.873044  13471.696015    82117   
            BaseC100         2153370  12.397070  41459.913382   358916   
            BnRelu6C10        380324   8.567702  14565.504910     8302   
            BnRelu6C100       270171   5.690567   9881.198002    97005   
            GeluNanC100       454051   8.677411  16667.342402   209870   
            MobileNetC100     270981  34.478974  25395.127205    46985   
            OINanRelu6C100    991718  16.713972  36506.225135   329682   
            OrderInvC100      746910  12.192618  27524.076486   240627   
carolp20002 BaseBnRelu6C10     93808   1.449205   3470.747342     3738   
            BaseC10          1543923   7.346595  36180.194687    28891   
            BnRelu6C10        350164   3.404977  12948.071153     6112   
            GeluNanC10        338133   6.861060  13859.380883    11150   
            OINanRelu6C10    1068213  15.272654  44740.957634    23097   
            OrderInvC10       579924   7.237897  23988.895107    16370   

                             acc_err  sdc  critical_sdc  count  
hostname    config                                              
carolm20001 BaseBnRelu6C100  1375858  274             6    274  
            BaseC10           259842  138             4    138  
            BaseC100         1738607  249            14    249  
            BnRelu6C100      2571379  364             8    364  
            GeluNanC100      1121898  181             8    181  
            MobileNetC100     161645  219             4    219  
            OINanRelu6C10      75209  104             5    104  
            OINanRelu6C100   1528901  242             7    242  
            OrderInvC100      876376  175             8    175  
carolp20001 BaseBnRelu6C100   135903  136             0    136  
            BaseC100          929259  502            18    502  
            BnRelu6C10         20386  153             0    153  
            BnRelu6C100       176851  138             0    138  
            GeluNanC100       414405  215             5    215  
            MobileNetC100     124144  277             4    277  
            OINanRelu6C100    582884  374             8    374  
            OrderInvC100      439031  300             4    300  
carolp20002 BaseBnRelu6C10      7255   38             0     38  
            BaseC10            82885  280             4    280  
            BnRelu6C10          9396   90             0     90  
            GeluNanC10         34135  161             4    161  
            OINanRelu6C10      36377  354             3    354  
            OrderInvC10        33362  168             4    168

In [11]:
grouped_df[["sdc", "critical_sdc", "ker_err", "count"]].to_csv("avf.csv")

In [12]:
grouped_df[["sdc", "critical_sdc", "acc_time"]]

sdc  critical_sdc      acc_time
hostname    config                                          
carolm20001 BaseBnRelu6C100  274             6   2929.383724
            BaseC10          138             4   2271.300349
            BaseC100         249            14   2379.867973
            BnRelu6C100      364             8   5291.821821
            GeluNanC100      181             8   2004.163502
            MobileNetC100    219             4   3043.894867
            OINanRelu6C10    104             5   1430.645173
            OINanRelu6C100   242             7   4160.807624
            OrderInvC100     175             8   2905.855822
carolp20001 BaseBnRelu6C100  136             0  13471.696015
            BaseC100         502            18  41459.913382
            BnRelu6C10       153             0  14565.504910
            BnRelu6C100      138             0   9881.198002
            GeluNanC100      215             5  16667.342402
            MobileNetC100    277             4  25395.127205
            OINanRelu6C100   374             8  36506.225135
            OrderInvC100     300             4  27524.076486
carolp20002 BaseBnRelu6C10    38             0   3470.747342
            BaseC10          280             4  36180.194687
            BnRelu6C10        90             0  12948.071153
            GeluNanC10       161             4  13859.380883
            OINanRelu6C10    354             3  44740.957634
            OrderInvC10      168             4  23988.895107

In [13]:
grouped_df["sdc_rate"] = grouped_df["sdc"] / grouped_df["acc_time"]
grouped_df["critical_sdc_rate"] = grouped_df["critical_sdc"] / grouped_df["acc_time"]

In [14]:
grouped_df

it   ker_time      acc_time  ker_err  \
hostname    config                                                       
carolm20001 BaseBnRelu6C100    44976  20.427763   2929.383724   482897   
            BaseC10            52690   6.720437   2271.300349    39879   
            BaseC100           54502  13.383705   2379.867973   420120   
            BnRelu6C100        80810  26.794611   5291.821821   599096   
            GeluNanC100        30643  12.635346   2004.163502   318669   
            MobileNetC100      15205  51.839837   3043.894867    35592   
            OINanRelu6C10      19859   7.945360   1430.645173    23266   
            OINanRelu6C100     63855  17.464981   4160.807624   407952   
            OrderInvC100       44598  12.336246   2905.855822   278677   
carolp20001 BaseBnRelu6C100   369035   5.873044  13471.696015    82117   
            BaseC100         2153370  12.397070  41459.913382   358916   
            BnRelu6C10        380324   8.567702  14565.504910     8302   
            BnRelu6C100       270171   5.690567   9881.198002    97005   
            GeluNanC100       454051   8.677411  16667.342402   209870   
            MobileNetC100     270981  34.478974  25395.127205    46985   
            OINanRelu6C100    991718  16.713972  36506.225135   329682   
            OrderInvC100      746910  12.192618  27524.076486   240627   
carolp20002 BaseBnRelu6C10     93808   1.449205   3470.747342     3738   
            BaseC10          1543923   7.346595  36180.194687    28891   
            BnRelu6C10        350164   3.404977  12948.071153     6112   
            GeluNanC10        338133   6.861060  13859.380883    11150   
            OINanRelu6C10    1068213  15.272654  44740.957634    23097   
            OrderInvC10       579924   7.237897  23988.895107    16370   

                             acc_err  sdc  critical_sdc  count  sdc_rate  \
hostname    config                                                         
carolm20001 BaseBnRelu6C100  1375858  274             6    274  0.093535   
            BaseC10           259842  138             4    138  0.060758   
            BaseC100         1738607  249            14    249  0.104628   
            BnRelu6C100      2571379  364             8    364  0.068785   
            GeluNanC100      1121898  181             8    181  0.090312   
            MobileNetC100     161645  219             4    219  0.071947   
            OINanRelu6C10      75209  104             5    104  0.072694   
            OINanRelu6C100   1528901  242             7    242  0.058162   
            OrderInvC100      876376  175             8    175  0.060223   
carolp20001 BaseBnRelu6C100   135903  136             0    136  0.010095   
            BaseC100          929259  502            18    502  0.012108   
            BnRelu6C10         20386  153             0    153  0.010504   
            BnRelu6C100       176851  138             0    138  0.013966   
            GeluNanC100       414405  215             5    215  0.012899   
            MobileNetC100     124144  277             4    277  0.010908   
            OINanRelu6C100    582884  374             8    374  0.010245   
            OrderInvC100      439031  300             4    300  0.010900   
carolp20002 BaseBnRelu6C10      7255   38             0     38  0.010949   
            BaseC10            82885  280             4    280  0.007739   
            BnRelu6C10          9396   90             0     90  0.006951   
            GeluNanC10         34135  161             4    161  0.011617   
            OINanRelu6C10      36377  354             3    354  0.007912   
            OrderInvC10        33362  168             4    168  0.007003   

                             critical_sdc_rate  
hostname    config                              
carolm20001 BaseBnRelu6C100           0.002048  
            BaseC10                   0.001761  
            BaseC100                  0.005883  
            BnRelu6C100              

In [15]:
grouped_df[["sdc", "critical_sdc", "acc_time", "sdc_rate", "critical_sdc_rate"]]

sdc  critical_sdc      acc_time  sdc_rate  \
hostname    config                                                       
carolm20001 BaseBnRelu6C100  274             6   2929.383724  0.093535   
            BaseC10          138             4   2271.300349  0.060758   
            BaseC100         249            14   2379.867973  0.104628   
            BnRelu6C100      364             8   5291.821821  0.068785   
            GeluNanC100      181             8   2004.163502  0.090312   
            MobileNetC100    219             4   3043.894867  0.071947   
            OINanRelu6C10    104             5   1430.645173  0.072694   
            OINanRelu6C100   242             7   4160.807624  0.058162   
            OrderInvC100     175             8   2905.855822  0.060223   
carolp20001 BaseBnRelu6C100  136             0  13471.696015  0.010095   
            BaseC100         502            18  41459.913382  0.012108   
            BnRelu6C10       153             0  14565.504910  0.010504   
            BnRelu6C100      138             0   9881.198002  0.013966   
            GeluNanC100      215             5  16667.342402  0.012899   
            MobileNetC100    277             4  25395.127205  0.010908   
            OINanRelu6C100   374             8  36506.225135  0.010245   
            OrderInvC100     300             4  27524.076486  0.010900   
carolp20002 BaseBnRelu6C10    38             0   3470.747342  0.010949   
            BaseC10          280             4  36180.194687  0.007739   
            BnRelu6C10        90             0  12948.071153  0.006951   
            GeluNanC10       161             4  13859.380883  0.011617   
            OINanRelu6C10    354             3  44740.957634  0.007912   
            OrderInvC10      168             4  23988.895107  0.007003   

                             critical_sdc_rate  
hostname    config                              
carolm20001 BaseBnRelu6C100           0.002048  
            BaseC10                   0.001761  
            BaseC100                  0.005883  
            BnRelu6C100               0.001512  
            GeluNanC100               0.003992  
            MobileNetC100             0.001314  
            OINanRelu6C10             0.003495  
            OINanRelu6C100            0.001682  
            OrderInvC100              0.002753  
carolp20001 BaseBnRelu6C100           0.000000  
            BaseC100                  0.000434  
            BnRelu6C10                0.000000  
            BnRelu6C100               0.000000  
            GeluNanC100               0.000300  
            MobileNetC100             0.000158  
            OINanRelu6C100            0.000219  
            OrderInvC100              0.000145  
carolp20002 BaseBnRelu6C10            0.000000  
            BaseC10                   0.000111  
            BnRelu6C10                0.000000  
            GeluNanC10                0.000289  
            OINanRelu6C10             0.000067  
            OrderInvC10               0.000167